# Analyzing RCT data with Precision Adjustment

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of six treatment groups. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data.
  

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

<ipython-input-2-c62976242921>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)


In [3]:
data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

,count
T4,
0,3354
1,1745


In [4]:
data.describe()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
count,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,...,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000,5099.000000
mean,10695.416356,1.368896,13.052952,12.281232,0.404001,0.121985,0.032555,0.007256,0.439694,0.012748,...,0.062954,0.110414,0.545009,0.109433,0.148068,0.109237,0.261032,0.218670,0.444205,0.342224
std,111.180503,1.898003,10.565602,10.362143,0.490746,0.327300,0.177487,0.084883,0.757622,0.112194,...,0.242903,0.313436,0.498019,0.312213,0.355202,0.311967,0.439240,0.413385,0.496926,0.474501
min,10404.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10600.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10698.000000,0.000000,11.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10796.000000,4.000000,25.000000,23.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,10880.000000,4.000000,52.000000,52.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Model
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicator,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, assuming the RCT assumptions hold. Within an RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However, we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.


We also consider an interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, assuming the RCT assumptions hold. Unlike the model without interactions, we are guaranteed to improve precision by considering the interactive model.

### Analysis

We consider

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

# Carry out covariate balance check


We first look at the coefficients individually with a $t$-test, and then we adjust the $p$-values to control for family-wise error.

The regression below is done using "type='HC1'" which computes the correct Eicker-Huber-White standard errors, instead of the classical non-robust formula based on homoscedasticity.

In [5]:
formula = ("0 ~ "
           "(female + black + othrace + C(dep) + q2 + q3 + q4 + q5 + q6 "
           "+ agelt35 + agegt54 + durable + lusd + husd)**2")
X = patsy.dmatrix(formula, data=data, return_type='dataframe')
X.shape

(5099, 120)

There is strong multicollinearity in the design matrix. In order to address this, we drop columns by thresholding using the QR decomposition.

In [ ]:
def qr_decomposition(x):
    # Get a set of columns with no linear dependence for smallest subset of observations
    Q, Rx = np.linalg.qr(x)
    ex = np.abs(np.diag(Rx))
    keep = np.where(ex > 1e-6)[0]
    xS = x.iloc[:, keep]

    return xS


xS = qr_decomposition(X)
xS.shape

In [ ]:
# individual t-tests
covariate_balance = sm.OLS(data["T4"], xS)
cb_fit = covariate_balance.fit(cov_type="HC1", use_t=True)
cb_fit.summary()

To test balance conditions, we employ the Holm-Bonferroni step-down method. With 100+ hypotheses, the family-wise type I error, or the probability of making at least one type I error treating all hypotheses independently, is close to 1. To control for this, we adjust p-values with the following procedure.

First, set $\alpha=0.05$ and denote the list of $n$ p-values from the regression with the vector $p$.

1. Sort $p$ from smallest to largest, so $p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(n)}$. Denote the corresponding hypothesis for $p_{(i)}$ as $H_{(i)}$.
2. For $i=1,\ldots, n$,
- If $$p_{(i)} > \frac{\alpha}{n-i+1} $$ Break the loop and do not reject any $H_{(j)}$ for $j \geq i$.
- Else reject $H_{(i)}$ if $$p_{(i)} \leq \frac{\alpha}{n-i+1} $$ Increment $i := i+1$.




In [ ]:
def holm_bonferroni(p, alpha=0.05):

    n = len(p)
    sig_beta = []

    for i in range(n):
        if np.sort(p)[i] > alpha / (n - i):
            break
        else:
            sig_beta.append(np.argsort(p)[i])
            i += 1

    return sig_beta


print("Significant Coefficients (Indices): ", holm_bonferroni(cb_fit.pvalues, alpha=0.05))

We see that that even though this is a randomized experiment, balance conditions are failed.
<!--
The holm method fails to reject any hypothesis. That is, we fail to reject the hypothesis that any coefficient is zero. Thus, in this randomized experiment, balance conditions are met. -->

# Model Specification

In [ ]:
# no adjustment (2-sample approach)
cl = smf.ols("np.log(inuidur1) ~ T4", data)

# adding controls
cra = smf.ols("np.log(inuidur1) ~ T4 + "
              "(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2",
              data)
# Omitted dummies: q1, nondurable, muld

cl_results = cl.fit(cov_type="HC1")
cl_results.summary()

In [ ]:
cra_results = cra.fit(cov_type="HC1")
cra_results.summary()

The interactive specification corresponds to the approach introduced in Lin (2013).

In [ ]:
# interactive regression model

ira_formula = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])]  # clean column names
X = (X - X.mean(axis=0))  # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * (" + "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC1")
ira_results.summary()

Next we try out partialling out with theoretically justified lasso. We use "plug-in" tuning with a theoretically valid choice of penalty $\lambda = 2 \cdot c \hat{\sigma} \sqrt{n} \Phi^{-1}(1-\alpha/2p)$, where $c>1$ and $1-\alpha$ is a confidence level, and $\Phi^{-1}$ denotes the quantile function. See book for more details.

We pull an analogue of R's rlasso. We find a Python code that tries to replicate the main function of hdm r-package. It was made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy). If not using colab, download its repository and copy this folder to your site-packages folder. In my case it is located here ***C:\Python\Python38\Lib\site-packages*** . We need to install this package ***pip install multiprocess***.

In [ ]:
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git

In [ ]:
import sys
sys.path.insert(1, "./hdmpy")

In [ ]:
# We wrap the package so that it has the familiar sklearn API
import hdmpy
from sklearn.base import BaseEstimator


class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])


def lasso_model():
    return RLasso(post=False)

In [ ]:
D = X['T4']
A = X.drop('T4', axis=1)  # contains X and interaction terms D*X
y = np.log(data["inuidur1"])

Dres = D - np.mean(D)
yres = y - lasso_model().fit(A, y).predict(A)

ira_lasso_results = sm.OLS(yres, Dres).fit(cov_type='HC1')
ira_lasso_results.summary()

### Results

In [ ]:
results = {
    "CL": [cl_results.params[cl.exog_names.index("T4")],
           cl_results.HC1_se[cl.exog_names.index("T4")]],
    "CRA": [cra_results.params[cra.exog_names.index("T4")],
            cra_results.HC1_se[cra.exog_names.index("T4")]],
    "IRA": [ira_results.params[ira.exog_names.index("T4")],
            ira_results.HC1_se[ira.exog_names.index("T4")]],
    "IRA w Lasso": [ira_lasso_results.params[0],
                    ira_lasso_results.bse[0]],
}
results = pd.DataFrame(results)
results.index = ["Estimate", "Standard Error"]
results
# for printing to LaTeX: print(results.to_latex())

Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From the IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


